# **Distribución de créditos por monto**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

In [2]:
df_financiamientos_2022_puebla = df_financiamientos_2022_puebla[['id','monto']]
df_financiamientos_2023_puebla = df_financiamientos_2023_puebla[['id','monto']]
df_financiamientos_2024_puebla = df_financiamientos_2024_puebla[['id','monto']]

## 2022

In [13]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_puebla['rango'] = df_financiamientos_2022_puebla['monto'].apply(asignar_segmento)
df_financiamientos_2022_puebla.head()


,id,monto,rango
0,1,21093.00,1
1,2,34318.00,1
2,3,43431.00,1
3,4,51998.00,1
4,5,94648.96,1


In [14]:
conteo_por_rango = df_financiamientos_2022_puebla['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,1705
1,2,747
2,3,334
3,5,190
4,4,128


In [15]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,1705
1,2,747
2,3,334
4,4,128
3,5,190


In [16]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
})
df_conteo_por_rango

,rango,conteo
0,< 1M,1705
1,< 2M,747
2,< 3M,334
4,< 4M,128
3,< 20M,190


In [17]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2022',  carpeta='assets/graficas')
fig.show()

## 2023

In [8]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_puebla['rango'] = df_financiamientos_2023_puebla['monto'].apply(asignar_segmento)
df_financiamientos_2023_puebla.head()


,id,monto,rango
0,1,19817.75,1
1,2,0.00,1
2,3,0.00,1
3,4,33908.00,1
4,5,277298.00,1


In [9]:
conteo_por_rango = df_financiamientos_2023_puebla['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,6481
1,2,2488
2,3,879
3,5,628
4,4,361


In [10]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,6481
1,2,2488
2,3,879
4,4,361
3,5,628


In [11]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
})
df_conteo_por_rango

,rango,conteo
0,< 1M,6481
1,< 2M,2488
2,< 3M,879
4,< 4M,361
3,< 20M,628


In [12]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2023',  carpeta='assets/graficas')
fig.show()

## 2024

In [3]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_puebla['rango'] = df_financiamientos_2024_puebla['monto'].apply(asignar_segmento)
df_financiamientos_2024_puebla.head()


,id,monto,rango
0,1,8574319.0,5
1,2,1633800.0,2
2,3,0.0,1
3,4,0.0,1
4,5,0.0,1


In [4]:
conteo_por_rango = df_financiamientos_2024_puebla['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,1478
1,2,572
2,3,223
3,5,142
4,4,82


In [5]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,1478
1,2,572
2,3,223
4,4,82
3,5,142


In [6]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    '1': '< 1M',
    '2': '< 2M',
    '3': '< 3M',
    '4': '< 4M',
    '5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
})
df_conteo_por_rango

,rango,conteo
0,< 1M,1478
1,< 2M,572
2,< 3M,223
4,< 4M,82
3,< 20M,142


In [7]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2024',  carpeta='assets/graficas')
fig.show()